In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here


#Initial Load boston_table_id = "opendatadbt.311.boston311"

sf311 = requests.get("https://data.sfgov.org/resource/vw6y-z8j6.json?$limit=50000&$where=requested_datetime>'2021-03-24'" ) 
sf311json = sf311.json()
cols = ['service_request_id','requested_datetime','closed_date','agency_responsible','service_name','service_details','address','neighborhoods_sffind_boundaries','status_description','lat','long']
sf_df = pd.DataFrame(sf311json, columns=cols)
sf_df

In [43]:
#Incremental load
sf_table_id = "opendatadbt.311.sf311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(requested_datetime), max(closed_date)
FROM `opendatadbt.311.sf311`
limit 10
"""

sfmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
sfmaxopendatedate = str(sfmaxopendatedataframe['f0_'][0])

sfmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
sfmaxclosedate = str(sfmaxclosedatedataframe['f1_'][0])


In [44]:

sf311 = requests.get("https://data.sfgov.org/resource/vw6y-z8j6.json?$limit=50000&$where=requested_datetime>'"+sfmaxopendatedate+"' OR closed_date between '" + sfmaxclosedate+ "' AND '" + current_datetime + "'") 
sf311json = sf311.json()
cols = ['service_request_id','requested_datetime','closed_date','agency_responsible','service_name','service_details','address','neighborhoods_sffind_boundaries','status_description','lat','long']
sf_df = pd.DataFrame(sf311json, columns=cols)
sf_df

,service_request_id,requested_datetime,closed_date,agency_responsible,service_name,service_details,address,neighborhoods_sffind_boundaries,status_description,lat,long
0,13562749,2021-03-05T13:34:27.000,2021-04-04T23:00:54.000,US Postal Service Maintenance Queue,Graffiti,Mail_box - Offensive,Intersection of CALIFORNIA ST and LAGUNA ST,Pacific Heights,Closed,37.78953808921,-122.42890646263
1,13655086,2021-03-31T08:34:00.000,2021-04-04T06:00:00.000,311 Service Request Queue - Hold,311 External Request,Other,Intersection of CESAR CHAVEZ ST and VERMONT ST,Potrero Hill,Closed,37.749477386475,-122.402862548828
2,13562186,2021-03-05T12:03:29.000,2021-04-04T23:00:54.000,US Postal Service Maintenance Queue,Graffiti,Mail_box - Not_Offensive,"2355 POLK ST, SAN FRANCISCO, CA, 94109",Polk Gulch,Closed,37.798548185792,-122.42242882823
3,13562445,2021-03-05T12:45:30.000,2021-04-04T23:00:54.000,US Postal Service Maintenance Queue,Graffiti,Mail_box - Not_Offensive,"3644 19TH ST, SAN FRANCISCO, CA, 94110",Mission Dolores,Closed,37.760044927744,-122.424863269667
4,13562756,2021-03-05T13:35:32.000,2021-04-04T23:00:54.000,US Postal Service Maintenance Queue,Graffiti,Mail_box - Not_Offensive,"145 CAPP ST, SAN FRANCISCO, CA, 94110",Mission,Closed,37.7644529,-122.4189439
5,13650623,2021-03-30T08:18:00.000,2021-04-04T05:47:00.000,311 Service Request Queue - Hold,Graffiti,Sidewalk_in_front_of_property - Not_Offensive,"46 CONKLING ST, SAN FRANCISCO, CA, 94124",Silver Terrace,Closed,37.735787937548,-122.402103497778
6,13652397,2021-03-30T12:55:00.000,2021-04-04T08:09:00.000,DPW Ops Queue,Street and Sidewalk Cleaning,Human or Animal Waste,"705 BROADWAY, SAN FRANCISCO, CA, 94133",Chinatown,Closed,37.797458648682,-122.408805847168
7,13654448,2021-03-30T22:34:54.000,2021-04-04T02:53:00.000,DPW Ops Queue,Encampments,Encampment Cleanup,"145 TURK ST, SAN FRANCISCO, CA, 94102",Tenderloin,Closed,37.7829564,-122.4113758
8,13654570,2021-03-31T04:22:00.000,2021-04-04T02:44:00.000,DPW Ops Queue,Street and Sidewalk Cleaning,Other Loose Garbage,"393 EDDY ST, SAN FRANCISCO, CA, 94102",Tenderloin,Closed,37.783704618131,-122.414077687742
9,13656226,2021-03-31T11:44:00.000,2021-04-04T02:59:00.000,DPW - Bureau of Street Environmental Services - G,Street and Sidewalk Cleaning,City_garbage_can_overflowing,"40 CYRIL MAGNIN ST, SAN FRANCISCO, CA, 94102",Downtown / Union Square,Closed,37.7843073,-122.4081964


In [45]:
sf_table_id = "opendatadbt.311.sf311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('sf311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    sf_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [3]:
SF_table_id = "opendatadbt.311.sf311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "SF"
FROM `opendatadbt.311.sf311`

"""

sfdatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

sfdatacount = sfdatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

sf_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    sfdatacount, table_ref, job_config=job_config
).result()  # Make an API request.